In [14]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [15]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("challenge_data/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Rain (inches),Snow (inches)
0,0,Bluff,-46.60,168.33,49.55,77,93,19.82,light rain,NZ,0.15,0
1,1,Hobart,-42.88,147.33,39.00,100,0,5.82,clear sky,AU,0.00,0
2,2,Santa Rosa,14.31,121.11,84.99,78,100,4.00,light rain,PH,0.33,0
3,3,East London,-33.02,27.91,77.94,33,98,14.23,overcast clouds,ZA,0.00,0
4,4,Uruzgan,32.83,66.00,68.07,20,0,1.63,clear sky,AF,0.00,0


In [16]:
# Ask the customer to add a minimum and maximum temperature value and precipitation.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining= str(input("Do you want it to be raining? (yes/no) "))
snowing= str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [17]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                      (city_data_df["Rain (inches)"] <= 0) & \
                                      (city_data_df["Snow (inches)"]<= 0)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Rain (inches),Snow (inches)
3,3,East London,-33.02,27.91,77.94,33,98,14.23,overcast clouds,ZA,0.0,0
11,11,Jamestown,42.10,-79.24,84.20,62,1,4.70,clear sky,US,0.0,0
12,12,Hithadhoo,-0.60,73.08,85.21,68,100,3.51,overcast clouds,MV,0.0,0
16,16,Lorengau,-2.02,147.27,82.31,77,87,9.66,overcast clouds,PG,0.0,0
20,20,Tiznit,29.58,-9.50,82.67,28,0,9.69,clear sky,MA,0.0,0
21,21,Victoria,22.29,114.16,86.00,74,100,11.01,overcast clouds,HK,0.0,0
32,32,Sangamner,19.57,74.22,78.12,75,100,7.54,overcast clouds,IN,0.0,0
41,41,Ixtapa,20.70,-105.20,82.99,78,75,2.24,broken clouds,MX,0.0,0
44,44,Cocorit,27.58,-109.97,89.60,43,90,14.99,overcast clouds,MX,0.0,0
49,49,Kaspiyskiy,45.39,47.37,86.58,43,65,6.08,broken clouds,RU,0.0,0


In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,East London,ZA,77.94,overcast clouds,-33.02,27.91,
11,Jamestown,US,84.20,clear sky,42.10,-79.24,
12,Hithadhoo,MV,85.21,overcast clouds,-0.60,73.08,
16,Lorengau,PG,82.31,overcast clouds,-2.02,147.27,
20,Tiznit,MA,82.67,clear sky,29.58,-9.50,
21,Victoria,HK,86.00,overcast clouds,22.29,114.16,
32,Sangamner,IN,78.12,overcast clouds,19.57,74.22,
41,Ixtapa,MX,82.99,broken clouds,20.70,-105.20,
44,Cocorit,MX,89.60,overcast clouds,27.58,-109.97,
49,Kaspiyskiy,RU,86.58,broken clouds,45.39,47.37,


In [19]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [21]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,East London,ZA,77.94,overcast clouds,-33.02,27.91,Tu Casa
11,Jamestown,US,84.20,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
12,Hithadhoo,MV,85.21,overcast clouds,-0.60,73.08,Scoop Guest House
16,Lorengau,PG,82.31,overcast clouds,-2.02,147.27,Lorengau Harbourside Hotel
20,Tiznit,MA,82.67,clear sky,29.58,-9.50,
21,Victoria,HK,86.00,overcast clouds,22.29,114.16,Mini Hotel Central
32,Sangamner,IN,78.12,overcast clouds,19.57,74.22,Panchavati Group of Hotels & Motels
41,Ixtapa,MX,82.99,broken clouds,20.70,-105.20,Container Inn Puerto Vallarta
44,Cocorit,MX,89.60,overcast clouds,27.58,-109.97,motel kaba
49,Kaspiyskiy,RU,86.58,broken clouds,45.39,47.37,"Gostinitsa ""Gostevoy Dvor"""


In [23]:
# Create the output file (CSV).
output_data_file = "challenge_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file)

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [25]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))